In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

In [4]:
from market_value_predictor.preproc import manual_encoding, reduce_number_of_classes

In [5]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config; set_config(display='diagram')
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [6]:
df = pd.read_csv("../../raw_data/master_df_with_webscraping.csv")
df_train = pd.read_csv("../../raw_data/master_data_train.csv").drop(columns="Unnamed: 0")
df_test = pd.read_csv("../../raw_data/master_data_test.csv").drop(columns="Unnamed: 0")

# Preprocessing

In [7]:
from sklearn.preprocessing import FunctionTransformer

drop_nas = FunctionTransformer(lambda df: df.dropna(subset=list(df.select_dtypes(object).columns)))

feat_eng_player_traits = FunctionTransformer(lambda df: manual_encoding(df, "player_traits"))
feat_eng_player_tags = FunctionTransformer(lambda df: manual_encoding(df, "player_tags"))
feat_eng_player_positions = FunctionTransformer(lambda df: manual_encoding(df, "player_positions"))

dim_reduction_nationality = FunctionTransformer(lambda df: reduce_number_of_classes(df, "nationality", 50))
dim_reduction_league_name = FunctionTransformer(lambda df: reduce_number_of_classes(df, "league_name", 100))

cluster_team_position = FunctionTransformer(lambda df: cluster_team_position(df))

feat_eng = Pipeline([
    ("player_traits", feat_eng_player_traits),
    ("player_tags", feat_eng_player_tags),
    ("player_positions", feat_eng_player_positions),
    ("nationality", dim_reduction_nationality),
    ("league_name", dim_reduction_league_name),
    ("drop_nas", drop_nas),
#     ("team_position", cluster_team_position)
])

feat_eng

Pipeline(steps=[('player_traits',
                 FunctionTransformer(func=<function <lambda> at 0x7f08ee469ee0>)),
                ('player_tags',
                 FunctionTransformer(func=<function <lambda> at 0x7f08ee469f70>)),
                ('player_positions',
                 FunctionTransformer(func=<function <lambda> at 0x7f08ee4831f0>)),
                ('nationality',
                 FunctionTransformer(func=<function <lambda> at 0x7f08ee483280>)),
                ('league_name',
                 FunctionTransformer(func=<function <lambda> at 0x7f08ee483310>)),
                ('drop_nas',
                 FunctionTransformer(func=<function <lambda> at 0x7f08ee469dc0>))])

In [8]:
df_train_transformed = feat_eng.fit_transform(df_train)

In [9]:
df_test_transformed = feat_eng.transform(df_test)

# Pipeline

## Define lists of columns for pipeline

In [10]:
all_cats = list(df_train.select_dtypes(object).columns)

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

all_numerics = list(df_train.select_dtypes(include=numerics).columns)

all_numerics.remove("fee_cleaned")

encoded_columns = [elem for elem in all_numerics if "player_tags_" in elem] + [
    elem for elem in all_numerics if "player_positions_" in elem
] + [elem for elem in all_numerics if "player_traits_" in elem]

all_numerics_wo_encoded = []
for elem in all_numerics:
    if elem not in encoded_columns:
        all_numerics_wo_encoded.append(elem)

numericals_zero_impute = [
    "gk_diving", "gk_handling", "gk_kicking", "gk_reflexes", "gk_speed",
    "gk_positioning", "release_clause_eur"
]

numericals_mean_impute = []

for elem in all_numerics_wo_encoded:
    if elem not in numericals_zero_impute:
        numericals_mean_impute.append(elem)

## Define transformers

In [11]:
from tempfile import mkdtemp
from shutil import rmtree
cachedir = mkdtemp()

In [13]:
num_zero_tr = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("scaler", StandardScaler())
])

num_mean_tr = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

cat_tr = OneHotEncoder(handle_unknown='ignore')

## Build pipeline

In [45]:
preprocessor = ColumnTransformer(
    [("numerics_zero_imputing", num_mean_tr, numericals_zero_impute),
     ("numerics_mean_imputing", num_mean_tr, numericals_mean_impute),
     ("cat_tr", cat_tr, all_cats)],
remainder="passthrough")

pipe = Pipeline([
    #("feat_eng", feat_eng),
    ("preprocessing", preprocessor), 
    ("regressor", CatBoostRegressor(silent=True))], memory=cachedir)

pipe

Pipeline(memory='/tmp/tmpko17ktxj',
         steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerics_zero_imputing',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gk_diving', 'gk_handling',
                                                   'gk_kicking', 'gk_reflexes',
                                                   'gk_speed', 'gk_positioning',
                                                   'release_clause_eur']),
                                                 ('numerics_mean_imputing',
                                                  Pipe...
                                                   'movement_acceleration',
                                                   'movement_sprint_speed',
                                                   'movement_agility', ...]),
                                                 ('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['nationality', 'club_name',
                                                   'league_name',
                                                   'player_positions',
                                                   'preferred_foot',
                                                   'work_rate', 'body_type',
                                                   'real_face', 'player_tags',
                                                   'team_position',
                                                   'player_traits'])])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x7f85351d9490>)])

## Train pipeline    

In [36]:
X_train = df_train.drop(columns="fee_cleaned")
y_train = df_train[["fee_cleaned"]]
X_test = df_test.drop(columns="fee_cleaned")
y_test = df_test[["fee_cleaned"]]

In [46]:
pipe.fit(X_train, y_train)

Pipeline(memory='/tmp/tmpko17ktxj',
         steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerics_zero_imputing',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gk_diving', 'gk_handling',
                                                   'gk_kicking', 'gk_reflexes',
                                                   'gk_speed', 'gk_positioning',
                                                   'release_clause_eur']),
                                                 ('numerics_mean_imputing',
                                                  Pipe...
                                                   'movement_acceleration',
                                                   'movement_sprint_speed',
                                                   'movement_agility', ...]),
                                                 ('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['nationality', 'club_name',
                                                   'league_name',
                                                   'player_positions',
                                                   'preferred_foot',
                                                   'work_rate', 'body_type',
                                                   'real_face', 'player_tags',
                                                   'team_position',
                                                   'player_traits'])])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x7f85351d9490>)])

In [47]:
y_pred = pipe.predict(X_test)

In [48]:
r2_score(y_test, y_pred)

0.3924418139613042

## Grid search

In [40]:
X = df.drop(columns="fee_cleaned")
y = df[["fee_cleaned"]]

In [41]:
cross_val_score(pipe, X, y, cv=5, scoring='r2').mean()

Learning rate set to 0.049036
0:	learn: 11.5596796	total: 5.63ms	remaining: 5.63s
1:	learn: 11.4030716	total: 10.9ms	remaining: 5.45s
2:	learn: 11.2500514	total: 18.8ms	remaining: 6.25s
3:	learn: 11.0960953	total: 27.1ms	remaining: 6.75s
4:	learn: 10.9471980	total: 32.4ms	remaining: 6.46s
5:	learn: 10.8185935	total: 37.6ms	remaining: 6.22s
6:	learn: 10.6993727	total: 44ms	remaining: 6.24s
7:	learn: 10.5731898	total: 49ms	remaining: 6.07s
8:	learn: 10.4446417	total: 53.9ms	remaining: 5.94s
9:	learn: 10.3225757	total: 59.2ms	remaining: 5.86s
10:	learn: 10.2324274	total: 64.8ms	remaining: 5.83s
11:	learn: 10.1327336	total: 69.8ms	remaining: 5.75s
12:	learn: 10.0394689	total: 75.2ms	remaining: 5.71s
13:	learn: 9.9355965	total: 81.1ms	remaining: 5.71s
14:	learn: 9.8415856	total: 85.8ms	remaining: 5.63s
15:	learn: 9.7669990	total: 91.7ms	remaining: 5.64s
16:	learn: 9.6856226	total: 96.3ms	remaining: 5.57s
17:	learn: 9.6199615	total: 102ms	remaining: 5.58s
18:	learn: 9.5324476	total: 108ms	re

180:	learn: 7.0335979	total: 986ms	remaining: 4.46s
181:	learn: 7.0298708	total: 991ms	remaining: 4.46s
182:	learn: 7.0235729	total: 1000ms	remaining: 4.46s
183:	learn: 7.0193735	total: 1s	remaining: 4.46s
184:	learn: 7.0159592	total: 1.01s	remaining: 4.45s
185:	learn: 7.0129107	total: 1.02s	remaining: 4.45s
186:	learn: 7.0086867	total: 1.02s	remaining: 4.44s
187:	learn: 7.0018580	total: 1.02s	remaining: 4.43s
188:	learn: 6.9987278	total: 1.03s	remaining: 4.42s
189:	learn: 6.9918168	total: 1.04s	remaining: 4.42s
190:	learn: 6.9875167	total: 1.04s	remaining: 4.41s
191:	learn: 6.9727830	total: 1.05s	remaining: 4.4s
192:	learn: 6.9589321	total: 1.05s	remaining: 4.4s
193:	learn: 6.9557033	total: 1.06s	remaining: 4.4s
194:	learn: 6.9464563	total: 1.06s	remaining: 4.4s
195:	learn: 6.9305010	total: 1.07s	remaining: 4.39s
196:	learn: 6.9267343	total: 1.07s	remaining: 4.38s
197:	learn: 6.9138257	total: 1.08s	remaining: 4.37s
198:	learn: 6.9009608	total: 1.08s	remaining: 4.36s
199:	learn: 6.8974

355:	learn: 5.8897101	total: 1.98s	remaining: 3.58s
356:	learn: 5.8883201	total: 1.98s	remaining: 3.57s
357:	learn: 5.8812824	total: 1.99s	remaining: 3.57s
358:	learn: 5.8709829	total: 2s	remaining: 3.57s
359:	learn: 5.8696213	total: 2s	remaining: 3.56s
360:	learn: 5.8653879	total: 2.01s	remaining: 3.56s
361:	learn: 5.8640462	total: 2.02s	remaining: 3.55s
362:	learn: 5.8627323	total: 2.02s	remaining: 3.55s
363:	learn: 5.8614353	total: 2.03s	remaining: 3.55s
364:	learn: 5.8601620	total: 2.04s	remaining: 3.54s
365:	learn: 5.8589062	total: 2.04s	remaining: 3.54s
366:	learn: 5.8576699	total: 2.05s	remaining: 3.54s
367:	learn: 5.8473796	total: 2.05s	remaining: 3.53s
368:	learn: 5.8461632	total: 2.06s	remaining: 3.52s
369:	learn: 5.8449625	total: 2.07s	remaining: 3.52s
370:	learn: 5.8415161	total: 2.07s	remaining: 3.51s
371:	learn: 5.8403372	total: 2.08s	remaining: 3.51s
372:	learn: 5.8369550	total: 2.08s	remaining: 3.5s
373:	learn: 5.8350758	total: 2.09s	remaining: 3.5s
374:	learn: 5.833909

527:	learn: 5.1588842	total: 3.17s	remaining: 2.83s
528:	learn: 5.1573765	total: 3.18s	remaining: 2.83s
529:	learn: 5.1521065	total: 3.19s	remaining: 2.83s
530:	learn: 5.1431814	total: 3.19s	remaining: 2.82s
531:	learn: 5.1424193	total: 3.2s	remaining: 2.81s
532:	learn: 5.1411417	total: 3.21s	remaining: 2.81s
533:	learn: 5.1375997	total: 3.21s	remaining: 2.8s
534:	learn: 5.1328556	total: 3.22s	remaining: 2.8s
535:	learn: 5.1317632	total: 3.23s	remaining: 2.79s
536:	learn: 5.1238018	total: 3.23s	remaining: 2.79s
537:	learn: 5.1223167	total: 3.24s	remaining: 2.78s
538:	learn: 5.1213074	total: 3.25s	remaining: 2.78s
539:	learn: 5.1150429	total: 3.26s	remaining: 2.77s
540:	learn: 5.1085573	total: 3.26s	remaining: 2.77s
541:	learn: 5.1015546	total: 3.27s	remaining: 2.77s
542:	learn: 5.1001500	total: 3.28s	remaining: 2.76s
543:	learn: 5.0907712	total: 3.29s	remaining: 2.75s
544:	learn: 5.0881271	total: 3.29s	remaining: 2.75s
545:	learn: 5.0873779	total: 3.3s	remaining: 2.74s
546:	learn: 5.07

694:	learn: 4.6475405	total: 4.38s	remaining: 1.92s
695:	learn: 4.6438544	total: 4.39s	remaining: 1.92s
696:	learn: 4.6430765	total: 4.39s	remaining: 1.91s
697:	learn: 4.6398616	total: 4.4s	remaining: 1.9s
698:	learn: 4.6388124	total: 4.41s	remaining: 1.9s
699:	learn: 4.6352311	total: 4.42s	remaining: 1.89s
700:	learn: 4.6295565	total: 4.42s	remaining: 1.89s
701:	learn: 4.6249040	total: 4.43s	remaining: 1.88s
702:	learn: 4.6210283	total: 4.44s	remaining: 1.87s
703:	learn: 4.6195055	total: 4.44s	remaining: 1.87s
704:	learn: 4.6143104	total: 4.45s	remaining: 1.86s
705:	learn: 4.6138163	total: 4.46s	remaining: 1.86s
706:	learn: 4.6132377	total: 4.47s	remaining: 1.85s
707:	learn: 4.6121066	total: 4.47s	remaining: 1.84s
708:	learn: 4.6116483	total: 4.48s	remaining: 1.84s
709:	learn: 4.6070928	total: 4.49s	remaining: 1.83s
710:	learn: 4.6016004	total: 4.5s	remaining: 1.83s
711:	learn: 4.6010499	total: 4.5s	remaining: 1.82s
712:	learn: 4.5935094	total: 4.51s	remaining: 1.82s
713:	learn: 4.588

...

376:	learn: 5.2991297	total: 3.43s	remaining: 5.66s
377:	learn: 5.2897309	total: 3.44s	remaining: 5.66s
378:	learn: 5.2887977	total: 3.45s	remaining: 5.66s
379:	learn: 5.2879273	total: 3.46s	remaining: 5.65s
380:	learn: 5.2850080	total: 3.47s	remaining: 5.64s
381:	learn: 5.2841627	total: 3.48s	remaining: 5.63s
382:	learn: 5.2793113	total: 3.49s	remaining: 5.62s
383:	learn: 5.2784147	total: 3.5s	remaining: 5.61s
384:	learn: 5.2775851	total: 3.51s	remaining: 5.6s
385:	learn: 5.2720802	total: 3.52s	remaining: 5.6s
386:	learn: 5.2712484	total: 3.53s	remaining: 5.59s
387:	learn: 5.2689472	total: 3.54s	remaining: 5.58s
388:	learn: 5.2633500	total: 3.56s	remaining: 5.59s
389:	learn: 5.2584350	total: 3.57s	remaining: 5.58s
390:	learn: 5.2539270	total: 3.58s	remaining: 5.58s
391:	learn: 5.2531067	total: 3.59s	remaining: 5.57s
392:	learn: 5.2461812	total: 3.6s	remaining: 5.56s
393:	learn: 5.2420160	total: 3.61s	remaining: 5.55s
394:	learn: 5.2376151	total: 3.62s	remaining: 5.54s
395:	learn: 5.23

536:	learn: 4.6809541	total: 5.24s	remaining: 4.52s
537:	learn: 4.6762404	total: 5.25s	remaining: 4.51s
538:	learn: 4.6756002	total: 5.26s	remaining: 4.5s
539:	learn: 4.6723888	total: 5.27s	remaining: 4.49s
540:	learn: 4.6666348	total: 5.28s	remaining: 4.48s
541:	learn: 4.6608933	total: 5.29s	remaining: 4.47s
542:	learn: 4.6573182	total: 5.3s	remaining: 4.46s
543:	learn: 4.6567734	total: 5.32s	remaining: 4.46s
544:	learn: 4.6510575	total: 5.33s	remaining: 4.45s
545:	learn: 4.6464738	total: 5.35s	remaining: 4.45s
546:	learn: 4.6458508	total: 5.36s	remaining: 4.44s
547:	learn: 4.6452625	total: 5.37s	remaining: 4.43s
548:	learn: 4.6405302	total: 5.38s	remaining: 4.42s
549:	learn: 4.6347779	total: 5.39s	remaining: 4.41s
550:	learn: 4.6277246	total: 5.4s	remaining: 4.4s
551:	learn: 4.6225893	total: 5.42s	remaining: 4.39s
552:	learn: 4.6190689	total: 5.42s	remaining: 4.38s
553:	learn: 4.6143884	total: 5.44s	remaining: 4.38s
554:	learn: 4.6132121	total: 5.46s	remaining: 4.38s
555:	learn: 4.60

711:	learn: 4.1905720	total: 6.86s	remaining: 2.77s
712:	learn: 4.1866187	total: 6.87s	remaining: 2.76s
713:	learn: 4.1846159	total: 6.87s	remaining: 2.75s
714:	learn: 4.1794736	total: 6.88s	remaining: 2.74s
715:	learn: 4.1784442	total: 6.89s	remaining: 2.73s
716:	learn: 4.1758694	total: 6.89s	remaining: 2.72s
717:	learn: 4.1715135	total: 6.9s	remaining: 2.71s
718:	learn: 4.1681322	total: 6.91s	remaining: 2.7s
719:	learn: 4.1640438	total: 6.92s	remaining: 2.69s
720:	learn: 4.1611256	total: 6.92s	remaining: 2.68s
721:	learn: 4.1572650	total: 6.93s	remaining: 2.67s
722:	learn: 4.1548335	total: 6.94s	remaining: 2.66s
723:	learn: 4.1517463	total: 6.94s	remaining: 2.65s
724:	learn: 4.1479951	total: 6.95s	remaining: 2.64s
725:	learn: 4.1432007	total: 6.96s	remaining: 2.63s
726:	learn: 4.1396881	total: 6.97s	remaining: 2.62s
727:	learn: 4.1353326	total: 6.97s	remaining: 2.6s
728:	learn: 4.1292033	total: 6.98s	remaining: 2.59s
729:	learn: 4.1287370	total: 6.99s	remaining: 2.58s
730:	learn: 4.1

870:	learn: 3.8007352	total: 8.06s	remaining: 1.19s
871:	learn: 3.7949049	total: 8.07s	remaining: 1.18s
872:	learn: 3.7918985	total: 8.07s	remaining: 1.17s
873:	learn: 3.7879094	total: 8.09s	remaining: 1.17s
874:	learn: 3.7856847	total: 8.09s	remaining: 1.16s
875:	learn: 3.7844182	total: 8.1s	remaining: 1.15s
876:	learn: 3.7820642	total: 8.11s	remaining: 1.14s
877:	learn: 3.7774809	total: 8.12s	remaining: 1.13s
878:	learn: 3.7771391	total: 8.12s	remaining: 1.12s
879:	learn: 3.7754674	total: 8.13s	remaining: 1.11s
880:	learn: 3.7751452	total: 8.14s	remaining: 1.1s
881:	learn: 3.7705937	total: 8.14s	remaining: 1.09s
882:	learn: 3.7670229	total: 8.15s	remaining: 1.08s
883:	learn: 3.7660759	total: 8.16s	remaining: 1.07s
884:	learn: 3.7621045	total: 8.17s	remaining: 1.06s
885:	learn: 3.7617408	total: 8.18s	remaining: 1.05s
886:	learn: 3.7614054	total: 8.18s	remaining: 1.04s
887:	learn: 3.7573441	total: 8.19s	remaining: 1.03s
888:	learn: 3.7569886	total: 8.2s	remaining: 1.02s
889:	learn: 3.7

0.3798569808712514

In [49]:
pipe.get_params()

{'memory': '/tmp/tmpko17ktxj',
 'steps': [('preprocessing',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('numerics_zero_imputing',
                                    Pipeline(steps=[('imputer', SimpleImputer()),
                                                    ('scaler', StandardScaler())]),
                                    ['gk_diving', 'gk_handling', 'gk_kicking',
                                     'gk_reflexes', 'gk_speed', 'gk_positioning',
                                     'release_clause_eur']),
                                   ('numerics_mean_imputing',
                                    Pipeline(steps=[('imputer', SimpleImputer()),
                                                    ('scaler', StandardS...
                                     'skill_curve', 'skill_fk_accuracy',
                                     'skill_long_passing', 'skill_ball_control',
                                     'movement_acceleration',
        

In [43]:
param_grid = {
    "preprocessing__numerics_zero_imputing__scaler": [RobustScaler(), MinMaxScaler(), StandardScaler()],
    "preprocessing__numerics_mean_imputing__scaler": [RobustScaler(), MinMaxScaler(), StandardScaler()],
    'preprocessing__numerics_zero_imputing__imputer__strategy': ['constant', "mean"],
    'regressor__alpha': [0.01, 0.1, 1], 
    'regressor__l1_ratio': [0.2, 0.5, 0.8]
}

In [44]:
# grid_search = GridSearchCV(
#     pipe, 
#     param_grid=param_grid,
#     cv=5,
#     scoring="r2",
#     n_jobs=-1)

# grid_search.fit(X, y)
# print(grid_search.best_params_)
# print(grid_search.best_score_)